In [1]:
!rm -rf rapidsai-csp-utils
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 530, done.
remote: Counting objects: 100% (261/261), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 530 (delta 171), reused 129 (delta 94), pack-reused 269 (from 1)
Receiving objects: 100% (530/530), 170.56 KiB | 259.00 KiB/s, done.
Resolving deltas: 100% (273/273), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.4 MB/s eta 0:00:00
Installing RAPIDS remaining 24.6.* libraries
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 478.0/478.0 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 GB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.4/141.4 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 895.7/895.7 kB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import cuml
print(cuml.__version__)

24.06.01


In [17]:
from tensorflow.keras.datasets import mnist
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cuml.svm import SVC
from cuml.neighbors import KNeighborsClassifier
from cuml.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from google.colab import files

In [7]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [24]:
res_list = []

for data_count in [500,1000,5000,10000,30000,60000]:
  x_train = np.array([np.ravel(x.astype(np.float32)) / 255.0 for x in x_train])
  x_test = np.array([np.ravel(x.astype(np.float32)) / 255.0 for x in x_test])

  x_train = x_train[:data_count]
  y_train = y_train[:data_count]

  x_test = x_test[:data_count]
  y_test = y_test[:data_count]

  params_svm = {"C": [0.01, 0.1, 0.5, 1, 5, 10 , 50, 100], "kernel" : ["linear", "poly", "rbf"]}
  params_knn = {"n_neighbors": [1,2,3,4,5,6,7,8,9,10]}
  params_log = {"C": [0.01, 0.1, 0.5, 1, 5, 10 , 50, 100]}


  params = params_log
  model = LogisticRegression()

  gr_s_model = GridSearchCV(model, params, cv=5, verbose=2, scoring="accuracy")
  gr_s_model.fit(x_train, y_train)

  y_pred = gr_s_model.predict(x_test)
  score = accuracy_score(y_test, y_pred)

  res_list.append(

      {"model": model.__class__.__name__,
      "n": data_count,
      "accuracy": score,
      "Best Estimator" : gr_s_model.best_estimator_,
      "Best Parameter" :  gr_s_model.best_params_})



  print(data_count, gr_s_model.best_params_)

best_result = max(res_list, key=lambda x: x['accuracy'])

print(f"Best Model: {best_result['model']}")
print(f"Accuracy: {best_result['accuracy']}")
print(f"Best Estimator: {best_result['Best Estimator']}")
print(f"Best Params: {best_result['Best Parameter']}")

df = pd.DataFrame(res_list)
df.to_excel("param2.xlsx", index=False)
files.download("param2.xlsx")

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END .............................................C=0.01; total time=   0.0s
[CV] END .............................................C=0.01; total time=   0.0s
[CV] END .............................................C=0.01; total time=   0.0s
[CV] END .............................................C=0.01; total time=   0.0s
[CV] END .............................................C=0.01; total time=   0.0s
[CV] END ..............................................C=0.1; total time=   0.0s
[CV] END ..............................................C=0.1; total time=   0.0s
[CV] END ..............................................C=0.1; total time=   0.1s
[CV] END ..............................................C=0.1; total time=   0.1s
[CV] END ..............................................C=0.1; total time=   0.0s
[CV] END ..............................................C=0.5; total time=   0.1s
[CV] END ........................................

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>